<h1>Open haloscope</h1>

<h4>Imports</h4>

In [1]:
import sys, os
import time

import numpy as np
from scipy.signal import periodogram, welch

import matplotlib.pyplot as plt
# plot options
#plt.rc('text', usetex = True)
#plt.rc('font', family = 'serif', size=14)

import qcodes as qc
from qcodes.interactive_widget import experiments_widget
## Multidimensional scanning module
from qcodes.dataset import (
    Measurement,
    do0d,
    do1d,
    experiments,
    initialise_or_create_database_at,
    load_or_create_experiment,
    load_by_run_spec,
    plot_dataset
)

In [2]:
import lib.open_haloscope.experiment as experiment
from lib.open_haloscope.utils import OHUtils

In [3]:
haloscope_name = 'kakapo_haloscope'
haloscope_json = OHUtils.load_experiment_json(haloscope_name)

H = experiment.FermionicHaloscope(haloscope_json)

In [4]:
from lib.red.redpitaya_qcodes import Redpitaya
red = Redpitaya('redpitaya', address=H.experiment_parameters['redpitaya_eth0_ip'])

Connected to: REDPITAYA INSTR2020 (serial:0, firmware:01-02) in 0.39s


<h4>Test of the drivers</h4>

In [5]:
H.experiment_parameters

{'_parameters_Kakapo': 'Parameters used in the fermionic haloscope Kakapo',
 'f0': 5000000.0,
 'Q': 100,
 'An': 1e-08,
 'Ap': 1.0,
 'beta1': 1,
 'beta2': 1,
 'operating_frequency': 'MHz',
 'material_used': 'NiZn ferrite',
 'redpitaya_eth0_ip': 'TCPIP::169.254.33.148::5000::SOCKET',
 'redpitaya_wlan_ip': 'TCPIP::192.168.1.30::5000::SOCKET'}

In [6]:
H.initialise_haloscope([red])

Loading instrumentation
  redpitaya added to the experiment

Setup the radiofrequency lines
 inputs configured, trigger level = 0.0 V, decimation = 4
 resulting sampling frequency = 31.25 MHz
 output generators triggered, phase aligned and turned off

Starting UART communication
 testing sensors
 temperature = 297.06 K
 pressure = 1.16231 bar
 magnetic field = 2.294921875 V
 photoresistance = 0.01953125 V
 acceleration = (0.08, -0.55, 10.36) m/s^2

Configuring data storage.
 data are stored in C:\Users\nicol\Progetti\Open Haloscope\data\runs\2023-05-24
 QCodes station and database configured

Haloscope initialised. Good luck, dark matter hunter.


In [7]:
H.characterise()

Done.


In [8]:
stop

NameError: name 'stop' is not defined

<h3>QCodes</h3>

In [12]:
station = qc.Station()
station.add_component(red)

'redpitaya'

In [13]:
initialise_or_create_database_at("data/runs/testdb.db")

In [ ]:
test_exp = load_or_create_experiment(experiment_name="testexp",
                                     sample_name="data")

context_meas = Measurement(exp=test_exp, station=station, name='contextexample')

red.vna_start(2e6)
red.vna_stop(10e6)
red.vna_points(101)
red.vna_rbw(10e3)

red.vna_amplitude(0.001)
red.vna_averages(1)

do0d(red.TX1);

In [ ]:
dataset_1d = load_by_run_spec(experiment_name='testexp', captured_run_id=1)
dataset_1d

In [ ]:
plot_dataset(dataset_1d)

In [17]:
from qcodes.tests.instrument_mocks import DummyInstrument
chrono2 = DummyInstrument('chrono2', gates=['t0', 't1'])

station.add_component(chrono2)

'chrono2'

In [24]:
do0d(red.acceleration)

Starting experimental run with id: 7. Using 'qcodes.dataset.do0d'


An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-01884b19b525;
Traceback:
Traceback (most recent call last):
  File "C:\Users\nicol\miniconda3\envs\ohal\Lib\site-packages\qcodes\dataset\dond\do_0d.py", line 83, in do0d
    datasaver.add_result(*process_params_meas(param_meas, use_threads=use_threads))
  File "C:\Users\nicol\miniconda3\envs\ohal\Lib\site-packages\qcodes\dataset\measurements.py", line 230, in add_result
    self.dataset._enqueue_results(results_dict)
  File "C:\Users\nicol\miniconda3\envs\ohal\Lib\site-packages\qcodes\dataset\data_set.py", line 1284, in _enqueue_results
    self.cache.add_data(new_results)
  File "C:\Users\nicol\miniconda3\envs\ohal\Lib\site-packages\qcodes\dataset\data_set_cache.py", line 124, in add_data
    ) = append_shaped_parameter_data_to_existing_arrays(
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nicol\miniconda3\envs\ohal\Lib\site-packages\qcodes\dataset\data_set_cache.py", line 297, in append

ValueError: could not broadcast input array from shape (3,) into shape (1,)

In [23]:
dataset_1d = load_by_run_spec(experiment_name='testexp', captured_run_id=6)
plot_dataset(dataset_1d)

Trying to mark a run completed that was already completed.


ValueError: could not broadcast input array from shape (30,) into shape (10,)

<h3>VNA</h3>

In [ ]:
red.vna_start(2e6)
red.vna_stop(10e6)
red.vna_points(11)
red.vna_rbw(10e3)

red.vna_amplitude(0.001)
red.vna_averages(1)

In [ ]:
mag1, phase1 = red.VNA1()
mag2, phase2 = red.VNA2()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,4))
ax.semilogy(red.frequency_axis()/1e6, mag1)
ax.semilogy(red.frequency_axis()/1e6, mag2)

In [ ]:
from scipy.optimize import curve_fit
def lorentzian(x, x0, a, gamma ):
    return a * gamma**2 / ( gamma**2 + ( x - x0 )**2)

In [ ]:
start = 400
stop = 470

x = red.frequency_axis()[start:stop] / 1e6
y = mag1[start:stop]**2 / np.max(mag1**2)

In [ ]:
popt, _ = curve_fit(lorentzian, x, y)
popt

In [ ]:
plt.plot(x, y, 'o')
plt.plot(x, lorentzian(x, *popt))

<h3>Waveforms</h3>

In [ ]:
# settings
# gen 1
red.OUT1_amplitude(1)
red.OUT1_frequency(5e6)
red.OUT1_status('ON')

# gen 2
red.OUT2_amplitude(0.01)
red.OUT2_frequency(2e5)
red.OUT2_status('OFF')

In [ ]:
# adc parameters
fs = H.sampling_frequency

In [ ]:
# acquisition
red.acquisition_length(10)
data = red.IN1()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,6))
n_start = 1
n_stop = 2**11

ax[0].imshow( data[:,n_start:n_stop], aspect='auto', origin='lower' )
ax[1].imshow( np.log10(periodogram( data[:,n_start:n_stop] )[1][1:]), aspect='auto', origin='lower' )

#ax[1].set_ylim([1e-8,1])

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,6))
n_start = 1
n_stop = 2**10
num = 0

ax[0].plot(data[num, n_start:n_stop])

f = periodogram(data[num, n_start:n_stop], fs=fs)[0]
rbw = f[1] - f[0]
m = rbw * periodogram(data[num, n_start:n_stop], window='hann', fs=fs)[1]
ax[1].semilogy(f, m)

In [ ]:
red.estimated_duty_cycle()